# Simulations with pixell

*Written by the ACT Collaboration*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/simonsobs/pixell_tutorials/blob/master/Pixell_simulations.ipynb)

---


This notebook, and the accompanying notebooks included in this set, are designed to help users who are new to working with [`pixell`](https://github.com/simonsobs/pixell/) get started with the package. As a set these notebooks will guide users through examples of how to read in and display maps, how to perform spherical harmonic transform and calculate simple spectra, how to transform the maps and how to study point sources in the maps.

The `pixell` library allows users to load,
manipulate and analyze maps stored in rectangular pixelization. It is
mainly targeted for use with maps of the sky (e.g. CMB intensity and polarization maps, stacks of 21 cm intensity maps, binned galaxy positions or shear) in cylindrical projection.

In this notebook we will use pixell functions to simulate millimeter-wave sky maps, including different components such as CMB, CMB lensing, Extragalactic Point Sources and Galaxy Clusters, and simplistic noise.

- [x] `enmap.randmap` (flat-sky Gaussian simulations, CMB and noise)

- [x] `curvedsky.randmap` (curved-sky Gaussian simulations, CMB and noise)

- [x] `lensing.lens_map` (flat-sky lensing operation)

- [x] `lensing.lens_map_curved` (curved-sky lensing operation)

- [x] `pointsrcs.sim_objects` (injecting point sources)




In [ ]:
# Install neccesary packages
!pip install pixell camb

In [ ]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt
from pixell import powspec
from pixell import utils
from pixell import enmap
from pixell import curvedsky
from pixell import lensing
from pixell import pointsrcs
from pixell import enplot

**Making a map from scratch**

For details on how to manipulate maps in pixell take a look at ["Pixell_map_manipulation.ipynb"](https://github.com/simonsobs/pixell_tutorials/blob/master/Pixell_map_manipulation.ipynb). First lets make a zeros map on a certain patch of the sky, let's use the region covering right ascension from 15 to 35 degrees and declination from -10 to -2 (you can try any patch). Here we need to define the vertices of a rectangle from left bottom to top right, these are RA0,DEC0 = 35,-10 and RA1,DEC1 = 15,-2 (think about the sky as going from right ascension 180 to -180 and declinations -90 to +90. We will use 0.5 arcminutes pixels in plate carrée projection, this means all pixels cover 0.5 arcminutes in latitude and longitude

In [ ]:
              # DEC0, RA0  DEC1 RA1
box = np.array([[-10, 35], [-2, 15]]) * utils.degree
shape, wcs = enmap.geometry(pos=box, res=0.5*utils.arcmin, proj='car')
# we will use only the temperature component or parameter stokes I
omap = enmap.zeros(shape, wcs)

In [ ]:
# we can use enplot and see what we have
enplot.pshow(omap, range=500, colorbar=True, downgrade=2)

looks like an empty map in the region that we wanted. We can also check the pixel size, since all pixels cover 0.5 in width and height we should have 8x60/0.5 = 960 pixels in height and 20x60/0.5 = 2400 in width


In [ ]:
omap.shape

This means that dimension = (number of stokes parameters, declination pixels, right ascension pixels)

**Gaussian realization of CMB**

There are two ways of simulating things, using flat sky approximation or curved sky. Let's start with flat sky, there is a handy function from `enmap` called `rand_map`, which draws a random realization from a power spectrum. To call this function we need to pass the following arguments:

`shape`: shape of the map that we want to simulate, in our case omap.shape

`wcs`: world coordinates of the map, in our case omap.wcs

`cov`: power spectrum that we want to include, more on this later

`scalar = False`: If we have polarization components, treat them as polarization

`seed = None`: The random seed that we want to use, if None if will generate one

`pixel_units = False`: The input power spectrum uses steradians

`iau = False`: The default polarization convention in the CMB community (sad)

`spin = [0,2]`: Apply a EB -> QU rotation

The three first argument are mandatory, but you may need to modify the others depending on your application (see the `pixell` documentation).

We need a power spectrum, specifically we want to simulate the CMB, for that we will use the Planck TT PS (you can try any other cosmology).

First we download the file in the format L TT EE BB TE


In [ ]:
!wget -O planck_lensedCls.dat https://raw.githubusercontent.com/simonsobs/nemo/main/nemo/data/planck_lensedCls.dat

We can plot the file to be sure if it contains the typical CMB power spectrum in the following way


In [ ]:
fileps = "planck_lensedCls.dat"
L, TT, EE, BB, TE = np.loadtxt(fileps, usecols=(0, 1, 2, 3, 4), unpack=True)
plt.loglog(L, TT)
# plt.yscale("log")
plt.xlabel(r"$\ell$")
plt.ylabel(r"$\ell (\ell+1) C_{\ell}^{TT}/(2 \pi)$")

In [ ]:
TT.shape

We can then pass this Planck TT power spectrum to the pixell function `powspec.read_spectrum`, so that the array is reshaped in the form that Pixell expects, and the 2$\pi/(\ell (\ell+1))$ factors are applied.

In [ ]:
ps = powspec.read_spectrum(fileps, scale=True, expand=None)

In [ ]:
ell = np.arange(len(ps[0]))
plt.semilogx(L, TE)
plt.semilogx(ell, ps[3] * (ell*(ell+1)) / (2*np.pi))

The pixell function `enmap.rand_map` will then take in the shape and wcs that we defined above, as well as the power spectrum from the cell above to make the corresponding map.

In [ ]:
# T only
cmbmap = enmap.rand_map(omap.shape, omap.wcs, cov=ps[0])
enplot.pshow(cmbmap, range=500, colorbar=True, downgrade=2)

If you run the previous code multiple times, you will see a different version each time, that's because each random realization is different but it is sampled from the same probability distribution (given by the power spectrum).

We can also make this polarized. We also need a 3-dimensional map to hold the simulation. We also need to change the `cov` parameter so that it includes all polarization components. We have a TT, EE, BB, and TE spectrum. We need to put this into a `(3, 3, nl)` array, rather than just an `(nl,)` array:

In [ ]:
nl = ps.shape[-1]
l = np.arange(nl)

ps_square = np.zeros((3, 3, nl))
print(ps_square.shape)

ps_square[0, 0] = ps[0]
ps_square[0, 1] = ps[3]
ps_square[1, 0] = ps[3]
ps_square[1, 1] = ps[1]
ps_square[2, 2] = ps[2]

plt.semilogy(l, l*(l+1)/2/np.pi*ps_square[0, 0], label='00')
plt.semilogy(l, l*(l+1)/2/np.pi*ps_square[1, 1], label='11')
plt.semilogy(l, l*(l+1)/2/np.pi*ps_square[2, 2], label='22')
plt.ylim(1e-3, 1e4)
plt.legend()
plt.show()

plt.plot(l, l*(l+1)/2/np.pi*ps_square[0, 1], label='01')
plt.legend()

Then we provide this square array as the `cov` parameter instead:

In [ ]:
# T, Q, U
cmbmap_pol = enmap.rand_map((3, *shape), wcs, cov=ps_square)
for i in range(3):
  enplot.pshow(cmbmap_pol[i], range=[500, 25, 25][i], colorbar=True, downgrade=2)

Now this is using flat sky approximation, so it uses the Fourier transform of the patch. If we want to account for the curvature of the sky, which is important for big patches we need to use the `curvedsky` module, in that module we can also find a `rand_map` function.

In [ ]:
# full sky goes from ra,dec +180, -90 to ra, dec -180, +90
# we can use bigger pixels such as 4 arcmin to avoid using too much memory
                #DEC0, RA0  DEC1 RA1
shape, wcs = enmap.fullsky_geometry(res=4*utils.arcmin, proj='car', variant='CC')
# we will use only the temperature component or parameter stokes I
omap = enmap.zeros((1, *shape), wcs)
# checking the map with a plot
enplot.pshow(omap, range=500, downgrade=4, ticks=15, colorbar=True)

This `curvedsky.rand_map` takes slightly different parameters than `enmap.rand_map`, in order they are

`shape`: shape of the map that we want to simulate, in our case omap.shape

`wcs`: world coordinates of the map, in our case omap.wcs

`ps`: power spectrum that we want to include. Can take an `(nl,)` shaped ps or a `(ncomp, ncomp, nl)` ps.

`lmax=None`: maximum multipole range to sample

`dtype=np.float32`: type of the resulting array

`seed=None`: number of the seed, if None it will generate one

`spin=[0,2]`: type of spin, in we have a IQU map it is [0,2], if we give as an input a single power spectrum it will take that into consideration to generate only one map

`method="auto"`: the default "auto" is fine

`verbose=False`: gives text information about the process

Note: it is typically sufficient to use single-precision maps (`dtype=np.float32`) which would speed up computation and use less memory. The default for this function is double precision (`dtype=np.float64`).

In [ ]:
# now simulating the CMB taking into account the curvature of the sky
# this may take some time since it uses spherical harmonics

# T only
cmbmap = curvedsky.rand_map(omap.shape, omap.wcs, ps=ps[0], lmax=3000, dtype=np.float32, verbose=True)
enplot.pshow(cmbmap, range=500, downgrade=4, ticks=15, colorbar=True)

Ok, it looks interesting. Notice the distortion at the poles; is this what you expected? To put it in context, consider this image of the earth in the same CAR projection:

![](https://upload.wikimedia.org/wikipedia/commons/thumb/8/83/Equirectangular_projection_SW.jpg/1920px-Equirectangular_projection_SW.jpg)

This makes sense now. Each pixel in the top row of pixels (and bottom row) are in fact the same physical point -- the pole! Using `enmap.rand_map` in this case might make a more uniform looking picture but it would be physically wrong -- we can't use the flat sky approximation when our range of declinations is large!

In [ ]:
# now simulating the CMB in this patch in the flat sky approximation
# NOTE: THIS IS WRONG
cmbmap = enmap.rand_map(omap.shape, omap.wcs, cov=ps[0])
enplot.pshow(cmbmap, range=500, colorbar=True, downgrade=4, ticks=15)

There are no distortions at the poles -- this is *wrong*. We need the distortions!

Just like the flatsky case, we can draw a polarized simulation. We again need a map to hold the 3 polarization components, and need to give it a power spectrum that puts the components in a square:

In [ ]:
# T, Q, U
cmbmap_pol = curvedsky.rand_map((3, *shape), wcs, ps=ps_square, lmax=3000, dtype=np.float32, verbose=True)
for i in range(3):
  enplot.pshow(cmbmap_pol[i], range=[500, 25, 25][i], downgrade=4, ticks=15, colorbar=True)

**Lensing**

To make flat-sky lensing simulations, we need unlensed CMB power spectra, as well as a lensing potential power spectrum. We obtain these from CAMB by running the following cells:

In [ ]:
import camb

pars = camb.set_params(H0=67.5, ombh2=0.022, omch2=0.122, mnu=0.06, omk=0, tau=0.06,
                       As=2e-9, ns=0.965, halofit_version='mead', lmax=4000)
pars.set_for_lmax(4000, lens_potential_accuracy=4)

# calculate results for these parameters
results = camb.get_results(pars)
powers = results.get_cmb_power_spectra(pars, CMB_unit='muK', raw_cl=True)
TT, EE, BB, TE = powers["unlensed_scalar"].T
ell = np.arange(len(TT))

# for simplicity assume lensing is uncorrelated with the CMB power spectra, but
# we could include correlations too as CAMB calculates them
PP = powers["lens_potential"][:, 0]

Before we create the lensed simulations, we need to package the CMB and lensing potential power spectra in a format that pixell will understand:

In [ ]:
ps_init = np.vstack((ell, TT, EE, TE, PP))
ps_init = np.atleast_2d(ps_init)
ps_init = powspec.expand_inds(np.array(ps_init[0],dtype=int), ps_init[1:])
ps_cmb = ps_init[:3]
ps_cmb = powspec.sym_expand(ps_cmb, scheme="diag", ncomp=3)
ps_lens = ps_init[3]

We are then ready to use the pixell function `enmap.rand_map` to make a $\phi$ map from the lensing potential power spectrum, as well as unlensed CMB maps from the CMB power spectra.

Let's be careful to go back to the small patch of sky for which we used `enmap.rand_map` in the beginning of this notebook, where the flat-sky limit is approximately valid:

In [ ]:
              # DEC0, RA0  DEC1 RA1
box = np.array([[-10, 35], [-2, 15]]) * utils.degree
shape, wcs = enmap.geometry(pos=box, res=0.5*utils.arcmin, proj='car')

phi_map = enmap.rand_map(shape, wcs, cov=ps_lens)
cmbmap = enmap.rand_map((3, *shape), wcs, cov=ps_cmb)

The pixell function `lensing.lens_map` will take in the CMB map we created above, as well as the gradient of the $\phi$ map and make a lensed version of the CMB maps. It has the following arguments:

`imap`: the unlensed map with shape `(..., ny, nx)` to be lensed

`grad_phi`: the map of the gradient of the lensing potential with shape `(2, ny, nx)`, obtained from `enmap.grad_phi`

`order=3`: related to the pixel-space interpolation (fine to keep)

`mode="spline"`: same as above

`border="cyclic"`: same as above

`trans=False`: if True, perform adjoint of lensing (not delensing)

`deriv=False`: whether to return derivatives of the interpolation

`h=1e-7`: finite difference size for the derivative

For standard lensing, we can accept all the defaults and just pass our unlensed map and lensing realization:

In [ ]:
grad_phi = enmap.grad(phi_map)
lensedcmb = lensing.lens_map(cmbmap, grad_phi, order=3, mode="spline", border="cyclic", trans=False, deriv=False, h=1e-7)

Let's take a look at the lensed CMB maps we just created, followed by the corresponding unlensed CMB map:

In [ ]:
enplot.pshow(lensedcmb[0], colorbar=True, downgrade=2, range=300)
enplot.pshow(cmbmap[0], colorbar=True, downgrade=2)

The effect of lensing is made more obvious by looking at the difference between the output and input (unlensed) CMB maps:

In [ ]:
enplot.pshow(lensedcmb[0] - cmbmap[0], colorbar=True, downgrade=2, range=50)

We can then compare the power spectra of these lensed maps to the lensed power spectra from CAMB. To calculate the power spectra of the lensed maps we created, we will first apodize the maps, and use the pixell functions `enmap.map2harm` and enmap.lbin to calculate the power spectra. If you need a refresher of this procedure, please refer to the notebook ["Fourier Operations with pixell"](https://github.com/simonsobs/pixell_tutorials/blob/master/Pixell_fourier_space_operations.ipynb).

Apodization is done as follows:

In [ ]:
apod_pix = 200 # number of pixels at the edge to apodize
taper = enmap.apod(cmbmap*0.0 + 1.0, apod_pix)

In [ ]:
for i in range(3):
  enplot.pshow((taper * lensedcmb)[i], downgrade=2, colorbar=True)

We can then compute the 2D FFT with `enmap.map2harm` and use `enmap.lbin` to bin the power spectra into 1D.

In [ ]:
harm_lensed = enmap.map2harm(taper * lensedcmb, normalize="phys")
w2 = np.mean(taper**2)
power = (harm_lensed[:, None] * np.conj(harm_lensed)).real/w2
cl, ls1 = power.lbin(bsize=40)

Here, we also load the CAMB lensed power spectra to compare to the lensed power spectra from the simulation we created in this notebook.

In [ ]:
powers_lensed_camb =  powers["lensed_scalar"].T

In [ ]:
dll_fac = (ls1*(ls1+1))/(2*np.pi)
camb_dll_fac = (ell*(ell+1))/(2*np.pi)

fig, ax = plt.subplots(2, 2)

ax[0,0].plot(ls1,cl[0, 0]*dll_fac,label="This notebook")
ax[0,0].plot(powers_lensed_camb[0]*camb_dll_fac,label="CAMB lensed")
ax[0,0].set_xlim(20, 4000)

ax[0,1].plot(ls1,cl[1, 1]*dll_fac,label="This notebook")
ax[0,1].plot(powers_lensed_camb[1]*camb_dll_fac,label="CAMB lensed")
ax[0,1].set_xlim(20, 4000)

ax[1,0].plot(ls1,cl[2, 2]*dll_fac,label="This notebook")
ax[1,0].plot(powers_lensed_camb[2]*camb_dll_fac,label="CAMB lensed")
ax[1,0].set_xlim(20, 4000)

ax[1,1].plot(ls1,cl[0, 1]*dll_fac,label="This notebook")
ax[1,1].plot(powers_lensed_camb[3]*camb_dll_fac,label="CAMB lensed")
ax[1,1].set_xlim(20, 4000)

ax[0,0].set_xlabel("$\ell$")
ax[0,0].set_ylabel("TT")
ax[0,1].set_xlabel("$\ell$")
ax[0,1].set_ylabel("EE")
ax[1,0].set_xlabel("$\ell$")
ax[1,0].set_ylabel("BB")
ax[1,1].set_xlabel("$\ell$")
ax[1,1].set_ylabel("TE")

plt.tight_layout()
plt.legend()
plt.show()

Nice! There are clearly some lensing-induced B-modes, but they don't have quite the right shape. This is likely due to the fact that we've done everything on the flat sky and/or haven't properly accounted for the mode-coupling due to the sky mask.

Fortunately, just as for the Gaussian realization from the CMB, `pixell` also supports full-sky lensing operations that accounts for spherical geometry. Because we can work on the full-sky, we won't need to apodize the map; however, this will not apply to a realistic simulation for ACT or SO data, which only observes a fraction of the sky.

As for the Gaussian realization, the curved-sky lensing operation `lensing.lens_map_curved` has new arguments:

`shape`: with `wcs`, the geometry of the output map

`wcs`: with shape, the geometry of the output map

`phi_alm`: the spherical harmonic realization of the lensing potential realization

`cmb_alm`: the spherical harmonics of the unlensed CMB realization

`phi_ainfo=None`: describes the spherical harmonic ordering convention for `phi` (you will know if you need to adjust this)

`maplmax=None`: deprecated

`dtype=np.float64`: output precision, needs to be double for accurate calculations (may crash otherwise)

`spin=[0,2]`: like in `curvedsky.rand_map`

`output="l"`: returns only the lensed CMB map. `lu` returns the lensed and unlensed CMB. More options for lensing maps in the [function documentation](https://github.com/simonsobs/pixell/blob/a61ca1a3ae99e4c8627c86a0fe2401d4bfe67d66/pixell/lensing.py#L134)

`geodesic=True`: slower but accurate, `False` for faster but less accurate

`verbose=False`: print helpful messages

`delta_theta=None`: can leave this as-is

In the below, besides supplying the required arguments, the only other important argument we change is to also return the unlensed CMB so we can compare:

In [ ]:
# first get a full-sky realization of the phi map and the polarized, unlensed cmb
# use low-resolution to reduce memory and runtime
shape, wcs = enmap.fullsky_geometry(res=4*utils.arcmin, proj='car')

# pixell needs the maps in spherical harmonics
phi_alm = curvedsky.rand_alm(ps=ps_lens, lmax=2700)
cmb_alm = curvedsky.rand_alm(ps=ps_cmb, lmax=2700)

# get the lensed cmb map
lensedcmb, cmbmap = lensing.lens_map_curved((3, *shape), wcs, phi_alm, cmb_alm, output="lu", verbose=True)

In [ ]:
ps_cmb[..., 3]

As before, let's take a look at the lensed CMB map, the unlensed CMB map, and their difference:

In [ ]:
enplot.pshow(lensedcmb[0], colorbar=True, downgrade=4, ticks=15)
enplot.pshow(cmbmap[0], colorbar=True, downgrade=4, ticks=15)
enplot.pshow(lensedcmb[0] - cmbmap[0], colorbar=True, downgrade=4, ticks=15)

Now we can take the full-sky power spectrum without a mask to see if the lensing produced the correct lensed CMB power spectrum. A nice benefit of this test compared to the flat-sky test is that we have more sky area and so the measured power spectrum is also less noisy:

In [ ]:
lensed_alm = curvedsky.map2alm(lensedcmb, lmax=2700)
cl = curvedsky.alm2cl(lensed_alm[:, None], lensed_alm)

In [ ]:
ell = np.arange(cl.shape[-1])
dll_fac = (ell*(ell+1))/(2*np.pi)

camb_ell = np.arange(powers_lensed_camb[0].shape[-1])
camb_dll_fac = (camb_ell*(camb_ell+1))/(2*np.pi)

fig, ax = plt.subplots(2, 2, figsize=(10, 6))

ax[0,0].plot(cl[0, 0]*dll_fac,label="This notebook")
ax[0,0].plot(powers_lensed_camb[0]*camb_dll_fac,label="CAMB lensed")
ax[0,0].set_xlim(20, 2700)

ax[0,1].plot(cl[1, 1]*dll_fac,label="This notebook")
ax[0,1].plot(powers_lensed_camb[1]*camb_dll_fac,label="CAMB lensed")
ax[0,1].set_xlim(20, 2700)

ax[1,0].plot(cl[2, 2]*dll_fac,label="This notebook")
ax[1,0].plot(powers_lensed_camb[2]*camb_dll_fac,label="CAMB lensed")
ax[1,0].set_xlim(20, 2700)

ax[1,1].plot(cl[0, 1]*dll_fac,label="This notebook")
ax[1,1].plot(powers_lensed_camb[3]*camb_dll_fac,label="CAMB lensed")
ax[1,1].set_xlim(20, 2700)

ax[0,0].set_xlabel("$\ell$")
ax[0,0].set_ylabel("TT")
ax[0,1].set_xlabel("$\ell$")
ax[0,1].set_ylabel("EE")
ax[1,0].set_xlabel("$\ell$")
ax[1,0].set_ylabel("BB")
ax[1,1].set_xlabel("$\ell$")
ax[1,1].set_ylabel("TE")

plt.tight_layout()
plt.legend()
plt.show()

Hooray, that worked well -- look at how much better the BB spectrum matches! Evidently the flat-sky lensing example was indeed biased either by its flat-sky treatment and/or simplistic handling of the mask when measuring the power spectrum. However, we should keep in mind that the lensing is not accurate out to the bandlimit -- we need to build-in some buffer to our analysis by lensing to a high bandlimit but only using larger scales (in this case, e.g., <2000).

**Point Sources**

In CMB maps we typically have extragalactic point sources emission and galaxy clusters through the Sunyaev Zel'dovich effect. The former are bright spots in the map and the latter it is a decrement for frequencies below 220 GHz and increment above 220 GHz, the effect it null approximately at 220 GHz.

To simulate point sources we need a profile, since these galaxies are usually unresolved points in the sky their profile is the telescope beam. For a 6-meter telescope the full width at half maximum of the beam is roughly 1.4 arcminutes at 150 GHz, we can simplify the profile as Gaussian.

Considering that a unit normalized Gaussian centered at zero has the form $\exp{\left( -\frac{r^2}{2\sigma^2} \right)}$ and the full width at half maximum is $\mathrm{FWHM} = 2 \sqrt{2 \ln(2)} \sigma$, we have the following functional form for the beam: $\exp{\left(-\frac{4\ln(2)r^2}{\mathrm{FWHM}^2}\right)}$

In [ ]:
# Generating a Gaussian beam
FWHM = 1.4
r = np.linspace(0, 60, 1000) # 60 arcminutes or 1 degree
B = np.exp(-(4*np.log(2)*r**2) / (FWHM**2))
plt.plot(r, B)
plt.xlim(0, 5)
plt.ylabel("Beam amplitude")
plt.xlabel("Radius (arcmin)")
# we need to pass the radius to radians
r *= 1/60 * np.pi / 180

Now we can generate sources, for that we will use the `pointsrcs` module, in specific the function `sim_objects`, this function requires the following:

`shape`: Shape of the map in which we will inject sources, in our case omap.shape

`wcs`: World coordinates of the map in which we will inject sources in our case omap.wcs

`poss`: Position of the sources that we want to inject in the form [dec,ra] where dec and ra are arrays of floats with the declination and right ascension of the sources in radians

`amp`: Amplitude of the sources that we want to inject, this is the peak value of the beam in temperature units as an array of floats

`profile`: The radial profile of the sources, in case of point sources this profile is the beam, but for example for cluster it could be a more extended profile, the profile is given as [r,B] where r is the radius in radians and B is the profile amplitude

`vmin=None`: The lowest value to simulate in map units, by default it takes 1e-3*amp.

`pixwin=False`: If we want to apply the pixel window function after simulating the objects.

A nice feature of this function is that it automatically handles the "distorted" geometry near the poles, so that radially-symmetric objects will appear horizontally stretched (as they should). This won't be evident in the small cutout below but it is important for a full-sky simulation:


In [ ]:
             # DEC0, RA0  DEC1 RA1
box = np.array([[-10, 35],[-2, 15]]) * utils.degree
shape, wcs = enmap.geometry(pos=box, res=0.5 * utils.arcmin, proj='car')
# we will use only the temperature component or parameter stokes I

# we will generate 100 sources
nsrc = 100
# we choose a logspace between 100 and 10000
amp = np.logspace(2.0, 4.0, nsrc)
# the position are random values inside omap
dec = np.random.uniform(-10, -2, nsrc) * np.pi / 180
ra = np.random.uniform(15, 35, nsrc)  *np.pi / 180

# we generate the sourcemap here
srcmap = pointsrcs.sim_objects(shape, wcs, [dec, ra], amp, [r, B],
                               min=np.min(amp)*1e-4)

# plotting the result
enplot.pshow(srcmap, range=500)

# adding to a cmb map
cmbmap = curvedsky.rand_map(shape, wcs, ps=ps[0])
enplot.pshow(srcmap + cmbmap, range=500)

**Simplistic Noise**

Finally we consider simple, additive noise that you can add to any of the previous realizations of the underlying "signal." Unlike the signal, which dies off quickly as a function of $\ell$, the noise is not bandlimited. Realistic noise in the maps is quite complicated (see https://arxiv.org/abs/2303.04180), so here we just show a simplified example. In fact, we've already used all of the tools.

We want to go back the case of using `enmap.rand_map` -- the flat-sky code -- to generate Gaussian realizations on the full-sky. This was unphysical for the signal, but for the noise, it is faster, sufficient, and in some ways better (again, the realization is not bandlimited).

We define a realistic noise power spectrum and level, and can also exploit that noise is ~symmetric in E and B to just perform a "scalar" rather than spin-2 transform:

In [ ]:
# make a polarized noise ps function
def T_and_P_noise_ps(ell, white_level=30, noise_ps_scaling=-4, T_knee=3000,
                     T_cap=300, P_knee=300, P_cap=100):
  """Get the temperature and polarization noise power spectra evaluated at ell.
  Follows this model:

  PS(ell) = white_level**2 * ((ell/knee)**scaling + 1), ell > cap
  PS(ell) = white_level**2 * ((cap/knee)**scaling + 1), ell <= cap

  Parameters
  ----------
  ell : (...) np.ndarray
    Angular scales.
  white_level : scalar
    Temperature white noise level in uK-arcmin. Polarization is this times
    sqrt(2).
  noise_ps_scaling : scalar
    Power-law scaling of the low-ell noise power spectra.
  T_knee : scalar
    Ell-knee of temperature power spectrum.
  T_cap : scalar
    Minimum ell at which the spectrum is capped.
  P_knee : scalar
    Ell-knee of polarization power spectrum.
  P_cap : scalar
    Minimum ell at which the spectrum is capped.

  Returns
  -------
  (3, ...) np.ndarray
    The polarization noise power spectra in T, Q, U. Assumed diagonal over
    polarization.
  """
  T = np.zeros_like(ell)
  mask = ell <= T_cap
  T[mask] = white_level**2 * ((T_cap/T_knee)**noise_ps_scaling + 1)
  T[~mask] = white_level**2 * ((ell[~mask]/T_knee)**noise_ps_scaling + 1)

  P = np.zeros_like(ell)
  mask = ell <= P_cap
  P[mask] = 2 * white_level**2 * ((P_cap/P_knee)**noise_ps_scaling + 1)
  P[~mask] = 2 * white_level**2 * ((ell[~mask]/P_knee)**noise_ps_scaling + 1)


  # convert to steradians (note, not square radians!). the below lines first
  # convert to square radians, then from square radians to steradians
  T *= utils.arcmin**2 * (4*np.pi / (np.pi * 2*np.pi))
  P *= utils.arcmin**2 * (4*np.pi / (np.pi * 2*np.pi))

  # put into square shape and return
  out = np.zeros((3, 3, *ell.shape), ell.dtype)
  out[0, 0] = T
  out[1, 1] = P
  out[2, 2] = P

  return out

Let's see what the default looks like as a function of $\ell$. This includes large-scale correlated noise from the atmosphere as would be observed by the SO LAT, but the SO SAT includes a half-wave plate and processing filters to mitigate this, so the noise spectra would need adjustment:

In [ ]:
ell = np.arange(10000, dtype=np.float64)
noise_ps = T_and_P_noise_ps(ell)
plt.loglog(ell, noise_ps[0, 0])
plt.loglog(ell, noise_ps[1, 1])

Let's make a convenience function to draw noise realizations, since this will also need to take into account the area of each pixel to keep the white-noise level consistent:

In [ ]:
def get_noise_sim(shape, wcs, seed=None, **T_and_P_noise_ps_kwargs):
  """Draw a noise realization from the constructed noise powre spectrum
  that also accounts for the smaller, and thus noisier, pixels near the
  poles.

  Parameters
  ----------
  shape : (ny, nx) tuple
    The footprint of the map.
  wcs : astropy.wcs.wcs.WCS
    The geometry of the map. Assumes units of degrees (the pixell
    default for wcs).
  seed : int or list of int
    Random seed.
  T_and_P_noise_ps_kwargs : dict
    Keyword arguments to be passed to T_and_P_noise_ps.

  Returns
  -------
  (3, ny, nx) enmap.ndmap
    Noise realization (polarized) drawn from T_and_P_noise_ps, with the
    correct white-noise level, correlated noise shape, and corrected for
    pixel areas.
  """
  # get the noise ps. for enmap.rand_map to work, this needs to be
  # evaluated at integer ells and cover all the angular scales in
  # out 2d fourier space
  ell = np.arange(0, np.ceil(enmap.modlmap(shape, wcs).max()) + 1)
  noise_ps = T_and_P_noise_ps(ell, **T_and_P_noise_ps_kwargs)

  # draw a noise realization
  noise_sim = enmap.rand_map((3, *shape), wcs, cov=noise_ps, seed=seed,
                             scalar=True)

  # normalize by pixel area. we want this in terms of fraction of a
  # "flat-sky" pixel
  pixsize_steradians = enmap.pixsizemap(shape, wcs, broadcastable=True)
  pix_area_deg = np.abs(np.prod(wcs.wcs.cdelt))
  pix_area_steradians = pix_area_deg * utils.degree**2
  frac_pixsize = pixsize_steradians / pix_area_steradians

  return noise_sim / np.sqrt(frac_pixsize)

Let's try it out on the full-sky:

In [ ]:
shape, wcs = enmap.fullsky_geometry(res=4*utils.arcmin, proj='car')

noise_sim = get_noise_sim(shape, wcs)

for i in range(3):
  enplot.pshow(noise_sim[i], downgrade=4, ticks=15, colorbar=True)

### Put it all together

Let's take all the above pieces and make a not-so-unrealistic simulation of data as would be observed by an actual CMB experiment:



1.   We'll draw a full-sky unlensed T, Q, U CMB realization and a full-sky realization of the lensing potential and use it to "lens" the CMB
2.   We'll add a new step to the above: convolve the resulting signal with
the instrumental beam so that it is consistent with the point sources
3.   We'll inject a sample of point sources with the same beam size. Assume they are not polarized
4.   We'll add a T, Q, U noise realization to the signal

This simulation omits quite a lot of real-world complexity in each of its parts --- we won't include any scan-synchronous signal, beam leakage or pixel window function, polarized or extended or "SZ-like" sources, or realistic noise --- but it is actually fairly representative.

We'll wrap it in a big convenience function. Play around with all the inputs or add even more arguments to make it more tunable!

(Exercise: An obvious way to streamline the below function would be to allow the user to pass pre-built signal or noise maps. That way someone could more efficiently add multiple noise realizations to the same signal realization, without rebuilding the same signal map each time)


In [ ]:
import healpy as hp

def get_signal_and_noise_sim(shape, wcs, lmax, ps_cmb, ps_lens, cmb_seed=None,
                             phi_seed=None, src_pos_seed=None, noise_seed=None,
                             beam_fwhm=1.4, nsrc=10000, white_level=30,
                             noise_ps_scaling=-4, T_knee=3000, T_cap=300,
                             P_knee=300, P_cap=100):
  """Draw a semi-realistic end-to-end sim including lensed CMB, temperature
  point sources, instrumental beam, and instrumental and atmospheric noise.

  Parameters
  ----------
  shape : (ny, nx) tuple
    The footprint of the map.
  wcs : astropy.wcs.wcs.WCS
    The geometry of the map. Assumes units of degrees (the pixell
    default for wcs).
  lmax : int
    maximum multipole for the signal
  ps_cmb : (3, 3, nl) np.ndarray
    CMB TEB power spectra with covariances
  ps_lens : (nl,) np.ndarray
    Lensing potential power spectrum
  cmb_seed : int or list of int
    Seed for unlensed CMB
  phi_seed : int or list of int
    Seed for lensing potential
  src_pos_seed : int or list of int
    Seed for point source locations
  noise_seed : int or list of int
    Seed for noise
  beam_fwhm : scalar
    Beam width in arcmin
  nsrc : int
    Number of point sources logarithmically distributed between
    100 and 10000 uK
  white_level : scalar
    Temperature white noise level in uK-arcmin. Polarization is this times
    sqrt(2).
  noise_ps_scaling : scalar
    Power-law scaling of the low-ell noise power spectra.
  T_knee : scalar
    Ell-knee of temperature power spectrum.
  T_cap : scalar
    Minimum ell at which the spectrum is capped.
  P_knee : scalar
    Ell-knee of polarization power spectrum.
  P_cap : scalar
    Minimum ell at which the spectrum is capped.

  Returns
  -------
  (3, ny, nx) enmap.ndmap
    The simulation
  """

  # get lensed CMB. assume lensing indep. of CMB (not true)
  phi_alm = curvedsky.rand_alm(ps=ps_lens, lmax=lmax, seed=phi_seed)
  cmb_alm = curvedsky.rand_alm(ps=ps_cmb, lmax=lmax, seed=cmb_seed)
  lensedcmb = lensing.lens_map_curved((3, *shape), wcs, phi_alm, cmb_alm,
                                      output="l", verbose=True)[0]

  # convolve it with the beam. fwhm in radians
  bl = hp.gauss_beam(beam_fwhm * utils.arcmin, lmax=lmax)
  lensedcmb_alm = curvedsky.map2alm(lensedcmb, lmax=lmax)
  curvedsky.almxfl(lensedcmb_alm, bl, out=lensedcmb_alm)
  curvedsky.alm2map(lensedcmb_alm, lensedcmb)

  # add srcs (already convolved with beam)
  r = np.linspace(0, 60, 1000) * utils.arcmin # 60 arcminutes or 1 degree
  B = np.exp(-4*np.log(2)*r**2 / (beam_fwhm*utils.arcmin)**2)
  amp = np.logspace(2.0, 4.0, nsrc)

  src_rng = np.random.default_rng(src_pos_seed)
  dec = src_rng.uniform(-90, 90, nsrc) * utils.degree
  ra = src_rng.uniform(-180, 180, nsrc)  * utils.degree
  srcmap = pointsrcs.sim_objects(shape, wcs, [dec, ra], amp, [r, B],
                                 vmin=np.min(amp)*1e-4)

  # and noise
  noisemap = get_noise_sim(shape, wcs, seed=noise_seed, white_level=white_level,
                           noise_ps_scaling=noise_ps_scaling, T_knee=T_knee,
                           T_cap=T_cap, P_knee=P_knee, P_cap=P_cap)

  out = lensedcmb + noisemap
  out[0] += srcmap
  return out

In [ ]:
totalsim = get_signal_and_noise_sim(shape, wcs, 2700, ps_cmb, ps_lens)

for i in range(3):
  enplot.pshow(totalsim[i], downgrade=4, ticks=15, colorbar=True)